In [66]:
import os

import pandas as pd
import numpy as np
import pandas_gbq

from google.cloud import bigquery
from google.oauth2 import service_account

import sqlite3

ModuleNotFoundError: No module named 'tqdm'

testing

In [56]:
service_path = ""
service_file = 'umt-msba-gg-key.json'

gbq_proj_id = 'umt-msba'

private_key = service_path + service_file   

credentials = service_account.Credentials.from_service_account_file(private_key)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)
for item in client.list_datasets() :
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


In [61]:
query="""

SELECT
 DATE(datetime) AS formatted_date,
 EXTRACT(hour FROM TIMESTAMP(datetime)) AS hour,
 ROUND(SUM(total), 2) AS spend,
 COUNT(DISTINCT(CONCAT(EXTRACT(date FROM datetime),register_no,emp_no,trans_no))) AS trans,
 SUM(CASE WHEN trans_status in ("R", "V") THEN -1 ELSE 1 END) AS items
FROM `umt-msba.transactions.transArchive_*`
WHERE department not in (0, 15) AND
(trans_status is null OR
trans_status in ("R","V", " "))
GROUP BY formatted_date, hour 
ORDER BY formatted_date, hour;

"""

In [62]:
conn = sqlite3.connect("reporting.db")

In [63]:
holder = pandas_gbq.read_gbq(query,project_id=gbq_proj_id,credentials=credentials)

c:\ADA\Wedge\wedge-project\.venv\Lib\site-packages\google\cloud\bigquery\table.py:2153: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [67]:
holder.head()

,formatted_date,hour,spend,trans,items
0,2010-01-01,9,1006.28,36,245
1,2010-01-01,10,3128.55,82,913
2,2010-01-01,11,4001.66,118,1108
3,2010-01-01,12,3886.51,124,1143
4,2010-01-01,13,4654.52,154,1365


In [68]:
holder.to_sql("date_hour",conn,if_exists="replace",index=False)

39330

In [69]:
conn.close()